## Campos semi estructurados

In [1]:
#Fijo el path

import os

# Parte de donde sea que esté el CWD actualmente
d = os.getcwd()

# Subir hasta encontrar la carpeta "Entregable"
while os.path.basename(d) != "Entregable":
    parent = os.path.dirname(d)
    if parent == d:   # Llegó al disco sin encontrar Entregable
        raise RuntimeError("No se encontró la carpeta Entregable hacia arriba.")
    d = parent

os.chdir(d)
print("CWD fijado en:", os.getcwd())

CWD fijado en: c:\Users\Mariana\Dropbox\Cursos\SoyHenry\DataEngineering\Modulo02-ModeladoDeDatosYBDRelacionales\ProyectoIntegrador\Entregable


In [6]:
import os
print(os.getcwd())

c:\Users\Mariana\Dropbox\Cursos\SoyHenry\DataEngineering\Modulo02-ModeladoDeDatosYBDRelacionales\ProyectoIntegrador


### Crar conexión y sesión

In [2]:
# Conexión y sesión
import os
import sys
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd

# Para poder importar src/*
#sys.path.append(os.path.abspath(".."))

from src.database.connection import DB

load_dotenv()
engine = DB.engine()
print ("Conectado a la base de datos")

Conectado a la base de datos


In [3]:
# Conexión a la base de dato y helpers
# 1) imports básicos
import pandas as pd
import re
import json

from src.database.connection import DB
from src.database.models import (
    Usuario, Categoria, Producto, Orden, DetalleOrden,
    DireccionEnvio, MetodoPago, OrdenMetodoPago,
    ResenaProducto, HistorialPago, Carrito
)

# 2) helper para traer una tabla como DataFrame

engine = DB.engine()

def load_table(table_name: str, schema: str = "staging", limit: int | None = 1000) -> pd.DataFrame:
    query = f"SELECT * FROM {schema}.{table_name}"
    if limit is not None:
        query += f" LIMIT {limit}"
    return pd.read_sql(query, con=engine)

# ejemplo rápido:
df_prod = load_table("productos")
df_prod.head()


,id,nombre,descripcion,precio,stock,categoria_id
0,1,Smartphone Galaxy A54,Teléfono inteligente con pantalla AMOLED y cám...,349.99,50,1
1,2,Laptop Dell Inspiron 15,Laptop para trabajo y estudio con procesador I...,799.00,30,1
2,3,Auriculares Bluetooth Sony,Auriculares inalámbricos con cancelación de ru...,129.99,100,1
3,4,Camiseta Básica Hombre,Camiseta de algodón 100% disponible en varias ...,14.99,200,2
4,5,Jeans Skinny Mujer,Jeans corte skinny de mezclilla azul.,39.99,150,2


## Buscar columnas con datos semiestructurados
La idea es escanear todas las tablas y marcar columnas TEXT/VARCHAR cuyo contenido:

- tenga muchos caracteres de separación (;, ,, |, :)
- parezca JSON ({}, [])
- tenga patrones tipo “lista” (muchos valores separados por el mismo delimitador)

In [4]:
TABLES = [
    "usuarios", "categorias", "productos", "ordenes", "detalle_ordenes",
    "direcciones_envio", "metodos_pago", "ordenes_metodos_pago",
    "resenas_productos", "historial_pagos", "carrito"
]

SEPARATORS = [",", ";", "|", "/"]

def scan_table_for_semi_structured(table_name: str, sample_rows: int = 1000):
    df = load_table(table_name, limit=sample_rows)
    report = []

    for col in df.columns:
        if df[col].dtype == "object":  # típicamente texto
            series = df[col].dropna().astype(str)
            if series.empty:
                continue

            sample = series.sample(min(len(series), 50), random_state=0)

            # heurísticas simples
            contains_json_like = sample.str.contains(r"^\s*[\{\[].*[\}\]]\s*$").mean()
            sep_stats = {
                sep: sample.str.contains(re.escape(sep)).mean()
                for sep in SEPARATORS
            }

            max_sep = max(sep_stats, key=sep_stats.get)
            max_sep_ratio = sep_stats[max_sep]

            if contains_json_like > 0.2 or max_sep_ratio > 0.3:
                report.append({
                    "tabla": table_name,
                    "columna": col,
                    "json_like_ratio": contains_json_like,
                    "main_sep": max_sep,
                    "main_sep_ratio": max_sep_ratio,
                })

    return pd.DataFrame(report)

all_reports = [scan_table_for_semi_structured(t) for t in TABLES]
semi_structured_report = pd.concat(all_reports, ignore_index=True)
semi_structured_report


,tabla,columna,json_like_ratio,main_sep,main_sep_ratio
0,categorias,descripcion,0.0,",",0.833333


## Aunque pareciera que el campo descripcion de la tabla categorias pudiera ser semiestructurado, al visualizarlo se ve que es una descripción.

No hay datos semiestructurados.